In [237]:
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import openai
from langchain.vectorstores import Pinecone
import os


In [238]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [240]:
#Function to read docuement
def read_doc(directory):
       dir_loader=PyPDFDirectoryLoader(directory)
       documents=dir_loader.load()
       return(documents)


In [241]:
docs=read_doc("documents/")
len(docs)

43

In [242]:
def chunk_data(docs,chunk_size=500,chunk_overlap=50):
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=50)
        doc=text_splitter.split_documents(docs)
        return doc

In [243]:

chunks=chunk_data(docs)
len(chunks)

234

In [244]:
len(chunks[0].page_content)


500

In [245]:
a=[]
for i in range(len(chunks)):
     #print(len(chunks[i].page_content))
     a.append(len(chunks[i].page_content))
     


In [246]:
max(a)

500

In [247]:
len(chunks[0].page_content)

500

In [258]:
embeddings=OpenAIEmbeddings(api_key="")

In [ ]:
vec=embeddings.embed_query(chunks[0].page_content)

In [ ]:
len(vec)

1536

In [ ]:
#Vector searchDb in pinecone
pinecone.init(
    api_key="",
    environment="")
index_name = 'semanticsearchtrial'

In [ ]:
import time
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
        pinecone.create_index(        
        name=index_name,
        dimension=1536,
        metric='cosine'

    )
    # wait a moment for the index to be fully initialized
time.sleep(1)

In [ ]:

index=Pinecone.from_documents(chunks,embeddings,index_name=index_name)

In [ ]:
##Cosine similarity to retrive results
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k)
    return matching_results


In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
llm=OpenAI(model_name="text-davinci-003",temperature=0.5,api_key="")
chain=load_qa_chain(llm,chain_type="stuff")


In [253]:
def findinfo(query):
    docsearch=retrieve_query(query,5)
    #print(docsearch)
    response=chain.run(input_documents=docsearch,question="Tell me brief in 5 line about {query}")
    return response

In [254]:
answer=findinfo("simple linear regression")

In [257]:
answer

' Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables. It assumes a linear relationship between the variables and aims to find the best-fitting line or hyperplane in multiple dimensions to make predictions. The objective in linear regression is to find the parameters (coefficients) of the linear model that minimize the difference between the predicted values and the actual values of the target variable. This is typically done using a cost function, such as the Mean Squared Error or the Sum of Squared Residuals. Linear regression can also be used in supervised machine learning to improve its predictive accuracy.'